## Description about this notebook
This notebook is designed to visually compare the performance of our three different YOLO NAS models (Small, Medium, Large) in terms of two key metrics: Mean Average Precision at 50% (MAP50) and Mean Average Precision from 50% to 95% (MAP50:95). We employed a supervision function for this evaluation. The assessment was carried out on a test dataset, which was not previously exposed to the models. This test dataset is unique as it comprises a new blend of Jersey and Holstein cow images, captured from two distinct camera perspectives: top and side views. This setup ensures a comprehensive evaluation of the models' ability to accurately detect and recognize objects in varied and realistic scenarios.

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
import logging
#####################################
# Set logging level to WARNING to ignore INFO messages
logging.basicConfig(level=logging.WARNING)

import os
os.environ['TORCH_HOME'] = '/home/mautushid/.torch'

#from ultralytics import NAS
os.chdir("..")
ROOT = os.getcwd()
print(ROOT)



/home/mautushid/Cowsformer


In [2]:
from models.nas import *
import pandas as pd
import matplotlib.pyplot as plt

The console stream is logged into /home/mautushid/sg_logs/console.log


[2024-03-04 14:28:53] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2024-03-04 14:28:54] WARNING - __init__.py - Failed to import pytorch_quantization
[2024-03-04 14:29:04] WARNING - __init__.py - Matplotlib created a temporary cache directory at /localscratch/2188895/matplotlib-0_uyk3py because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
[2024-03-04 14:29:06] INFO - font_manager.py - generated new fontManager
[2024-03-04 14:29:15] WARNING - calibrator.py - Failed to import pytorch_quantization
[2024-03-04 14:29:15] WARNING - export.py - Failed to import pytorch_quantization
[2024-03-04 14:29:15] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization


In [34]:

# Directory containing all the checkpoints
checkpoints_dir = ROOT + "/checkpoints"

# Function to sort and organize checkpoint paths into the specified format
def organize_checkpoints_corrected(checkpoints_dir):
    # Dictionary to hold the paths categorized by category, then by type
    categorized_paths = {}

    # Iterate through each subdirectory in the checkpoints directory
    for sub_dir in os.listdir(checkpoints_dir):
        # Construct the full path of the subdirectory
        full_sub_dir_path = os.path.join(checkpoints_dir, sub_dir)
        # Check if the current path is a directory
        if os.path.isdir(full_sub_dir_path):
            # Extract category, iteration, and type from the folder name
            parts = sub_dir.split('_')
            category = parts[0] 
            category2 = parts[-1]# e.g., breed
            size_type = parts[-3][-1]  # Assuming type is represented by the last letter before the category and iteration

            # Validate size_type to be 's', 'm', or 'l'
            if size_type not in ['s', 'm', 'l']:
                continue  # Skip if size_type does not match expected values
            
            # Iterate through the RUN_... directory inside the current subdirectory
            for run_dir in os.listdir(full_sub_dir_path):
                run_dir_path = os.path.join(full_sub_dir_path, run_dir)
                if os.path.isdir(run_dir_path):
                    # Construct the path to the ckpt_best.pth file inside the RUN_... directory
                    ckpt_path = os.path.join(run_dir_path, 'ckpt_best.pth')
                    if os.path.exists(ckpt_path):
                        # Initialize the nested dictionaries and lists as needed
                        if category not in categorized_paths:
                            categorized_paths[category] = {'s': [], 'm': [], 'l': []}
                        categorized_paths[category][size_type].append(ckpt_path)

    # Sort paths within each list
    for category in categorized_paths.keys():
        for size_type in categorized_paths[category].keys():
            categorized_paths[category][size_type].sort()

    return categorized_paths

# Call the function and print the organized checkpoint paths
# Directory containing all the checkpoints
checkpoints_dir = ROOT + "/checkpoints_cow200"

# Function to sort and organize checkpoint paths into the specified format
def organize_checkpoints_corrected(checkpoints_dir):
    # Dictionary to hold the paths categorized by category, then by type
    categorized_paths = {}

    # Iterate through each subdirectory in the checkpoints directory
    for sub_dir in os.listdir(checkpoints_dir):
        # Construct the full path of the subdirectory
        full_sub_dir_path = os.path.join(checkpoints_dir, sub_dir)
        # Check if the current path is a directory
        if os.path.isdir(full_sub_dir_path):
            # Extract category, iteration, and type from the folder name
            parts = sub_dir.split('_')
            category = parts[0]  # e.g., n10
            size_type = parts[-2][-1]  # Assuming type is represented by the last letter before the category and iteration

            # Validate size_type to be 's', 'm', or 'l'
            if size_type not in ['s', 'm', 'l']:
                continue  # Skip if size_type does not match expected values
            
            # Iterate through the RUN_... directory inside the current subdirectory
            for run_dir in os.listdir(full_sub_dir_path):
                run_dir_path = os.path.join(full_sub_dir_path, run_dir)
                if os.path.isdir(run_dir_path):
                    # Construct the path to the ckpt_best.pth file inside the RUN_... directory
                    ckpt_path = os.path.join(run_dir_path, 'ckpt_best.pth')
                    if os.path.exists(ckpt_path):
                        # Initialize the nested dictionaries and lists as needed
                        if category not in categorized_paths:
                            categorized_paths[category] = {'s': [], 'm': [], 'l': []}
                        categorized_paths[category][size_type].append(ckpt_path)

    # Sort paths within each list
    for category in categorized_paths.keys():
        for size_type in categorized_paths[category].keys():
            categorized_paths[category][size_type].sort()

    return categorized_paths

# Call the function and print the organized checkpoint paths
organized_paths = organize_checkpoints_corrected(checkpoints_dir)
for category, types in organized_paths.items():
    print(f"{category}:")
    for size_type, paths in types.items():
        print(f"  {size_type}:")
        for path in paths:
            print(f"    {path}")

organized_paths = organize_checkpoints_corrected(checkpoints_dir)
for category, types in organized_paths.items():
    print(f"{category}:")
    for size_type, paths in types.items():
        print(f"  {size_type}:")
        for path in paths:
            print(f"    {path}")


n32:
  s:
    /home/mautushid/Cowsformer/checkpoints/n32_yolo_nas_s_i1_all/RUN_20240229_131134_765755/ckpt_best.pth
    /home/mautushid/Cowsformer/checkpoints/n32_yolo_nas_s_i1_breed/RUN_20240301_021001_929576/ckpt_best.pth
    /home/mautushid/Cowsformer/checkpoints/n32_yolo_nas_s_i1_light/RUN_20240229_125926_722225/ckpt_best.pth
    /home/mautushid/Cowsformer/checkpoints/n32_yolo_nas_s_i1_s2t/RUN_20240229_131004_345526/ckpt_best.pth
    /home/mautushid/Cowsformer/checkpoints/n32_yolo_nas_s_i1_t2s/RUN_20240229_144817_124821/ckpt_best.pth
    /home/mautushid/Cowsformer/checkpoints/n32_yolo_nas_s_i2_all/RUN_20240229_230939_411706/ckpt_best.pth
    /home/mautushid/Cowsformer/checkpoints/n32_yolo_nas_s_i2_breed/RUN_20240301_110322_862373/ckpt_best.pth
    /home/mautushid/Cowsformer/checkpoints/n32_yolo_nas_s_i2_light/RUN_20240229_225430_498634/ckpt_best.pth
    /home/mautushid/Cowsformer/checkpoints/n32_yolo_nas_s_i2_s2t/RUN_20240229_230933_641404/ckpt_best.pth
    /home/mautushid/Cowsform

In [36]:
n10_s_path = organized_paths['n10']['s']
n10_m_path = organized_paths['n10']['m']
n10_l_path = organized_paths['n10']['l']

n25_s_path = organized_paths['n25']['s']
n25_m_path = organized_paths['n25']['m']
n25_l_path = organized_paths['n25']['l']

n50_s_path = organized_paths['n50']['s']
n50_m_path = organized_paths['n50']['m']
n50_l_path = organized_paths['n50']['l']

n100_s_path = organized_paths['n100']['s']
n100_m_path = organized_paths['n100']['m']
n100_l_path = organized_paths['n100']['l']

n200_s_path = organized_paths['n200']['s']
n200_m_path = organized_paths['n200']['m']
n200_l_path = organized_paths['n200']['l']

In [31]:

print(len(n10_m_path))
print(len(n10_l_path))

15
15


In [ ]:
print(len(n10_s_path))
print(len(n10_m_path))
print(len(n10_l_path))

print('------------------')

print(len(n25_s_path))
print(len(n25_m_path))
print(len(n25_l_path))

print('------------------')

print(len(n50_s_path))
print(len(n50_m_path))
print(len(n50_l_path))

print('------------------')
print(len(n100_s_path))
print(len(n100_m_path))
print(len(n100_l_path))

print('------------------')
print(len(n200_s_path))
print(len(n200_m_path))
print(len(n200_l_path))



In [ ]:
path_model_s = 'yolo_nas_s'
path_model_m = 'yolo_nas_m'
path_model_l = 'yolo_nas_l'
dir_train = "/home/mautushid/Cowsformer/data/cow200/yolov5/train"
dir_val = "/home/mautushid/Cowsformer/data/cow200/yolov5/val"
dir_test = "/home/mautushid/Cowsformer/data/cow200/yolov5/test"
name_task = "cow200"
data_yaml_path = "/home/mautushid/Cowsformer/data/cow200/yolov5/data.yaml"

def evaluate_models_s(finetuned_model_paths):
    models = []
    evaluation_results = []
    my_nas = Niche_YOLO_NAS(path_model_s, dir_train, dir_val, dir_test, name_task)
    
    # Load models
    for path in finetuned_model_paths:
        model = my_nas.load(path_model_s, path)
        models.append(model)
    
    # Evaluate models
    for model in models:
        metrics = my_nas.get_map_scores(model, data_yaml_path, "test")
        evaluation_results.append(metrics)
    
    return evaluation_results

def evaluate_models_m(finetuned_model_paths):
    models = []
    evaluation_results = []
    my_nas = Niche_YOLO_NAS(path_model_m, dir_train, dir_val, dir_test, name_task)
    
    # Load models
    for path in finetuned_model_paths:
        model = my_nas.load(path_model_m, path)
        models.append(model)
    
    # Evaluate models
    for model in models:
        metrics = my_nas.get_map_scores(model, data_yaml_path, "test")
        evaluation_results.append(metrics)
    
    return evaluation_results

def evaluate_models_l(finetuned_model_paths):
    models = []
    evaluation_results = []
    my_nas = Niche_YOLO_NAS(path_model_l, dir_train, dir_val, dir_test, name_task)
    
    # Load models
    for path in finetuned_model_paths:
        model = my_nas.load(path_model_l, path)
        models.append(model)
    
    # Evaluate models
    for model in models:
        metrics = my_nas.get_map_scores(model, data_yaml_path, "test")
        evaluation_results.append(metrics)
    
    return evaluation_results

In [ ]:
evaluattion_results_n10_s_path = evaluate_models_s(n10_s_path)
#evaluattion_results_n10_m_path = evaluate_models_m(n10_m_path)
#evaluattion_results_n10_l_path = evaluate_models_l(n10_l_path)

In [ ]:
evaluattion_results_n10_s_path = evaluate_models_s(n10_s_path)
evaluattion_results_n10_m_path = evaluate_models_m(n10_m_path)
evaluattion_results_n10_l_path = evaluate_models_l(n10_l_path)

evaluattion_results_n25_s_path = evaluate_models_s(n25_s_path)
evaluattion_results_n25_m_path = evaluate_models_m(n25_m_path)
evaluattion_results_n25_l_path = evaluate_models_l(n25_l_path)

evaluattion_results_n50_s_path = evaluate_models_s(n50_s_path)
evaluattion_results_n50_m_path = evaluate_models_m(n50_m_path)
evaluattion_results_n50_l_path = evaluate_models_l(n50_l_path)

evaluattion_results_n100_s_path = evaluate_models_s(n100_s_path)
evaluattion_results_n100_m_path = evaluate_models_m(n100_m_path)
evaluattion_results_n100_l_path = evaluate_models_l(n100_l_path)

evaluattion_results_n200_s_path = evaluate_models_s(n200_s_path)
evaluattion_results_n200_m_path = evaluate_models_m(n200_m_path)
evaluattion_results_n200_l_path = evaluate_models_l(n200_l_path)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming you have the actual evaluation data in these variables
# Replace these lines with your actual evaluation data
evaluation_results_n10_s_path = [{'mAP@50': np.random.uniform(0.4, 0.8)} for _ in range(8)]
evaluation_results_n10_m_path = [{'mAP@50': np.random.uniform(0.4, 0.8)} for _ in range(8)]
evaluation_results_n10_l_path = [{'mAP@50': np.random.uniform(0.4, 0.8)} for _ in range(8)]
evaluation_results_n25_s_path = [{'mAP@50': np.random.uniform(0.4, 0.8)} for _ in range(8)]
evaluation_results_n25_m_path = [{'mAP@50': np.random.uniform(0.4, 0.8)} for _ in range(8)]
evaluation_results_n25_l_path = [{'mAP@50': np.random.uniform(0.4, 0.8)} for _ in range(8)]
evaluation_results_n100_s_path = [{'mAP@50': np.random.uniform(0.4, 0.8)} for _ in range(8)]
evaluation_results_n100_m_path = [{'mAP@50': np.random.uniform(0.4, 0.8)} for _ in range(8)]
evaluation_results_n100_l_path = [{'mAP@50': np.random.uniform(0.4, 0.8)} for _ in range(8)]
evaluation_results_n50_s_path = [{'mAP@50': np.random.uniform(0.4, 0.8)} for _ in range(8)]
evaluation_results_n50_m_path = [{'mAP@50': np.random.uniform(0.4, 0.8)} for _ in range(8)]
evaluation_results_n50_l_path = [{'mAP@50': np.random.uniform(0.4, 0.8)} for _ in range(8)]
evaluation_results_n200_s_path = [{'mAP@50': np.random.uniform(0.4, 0.8)} for _ in range(8)]
evaluation_results_n200_m_path = [{'mAP@50': np.random.uniform(0.4, 0.8)} for _ in range(8)]
evaluation_results_n200_l_path = [{'mAP@50': np.random.uniform(0.4, 0.8)} for _ in range(8)]



# Collect all the results in a dictionary for processing
evaluation_results = {
    'n10_s': evaluattion_results_n10_s_path,
    'n10_m': evaluattion_results_n10_m_path,
    'n10_l': evaluattion_results_n10_l_path,
    'n25_s': evaluattion_results_n25_s_path,
    'n25_m': evaluattion_results_n25_m_path,
    'n25_l': evaluattion_results_n25_l_path,
    'n50_s': evaluattion_results_n50_s_path,
    'n50_m': evaluattion_results_n50_m_path,
    'n50_l': evaluattion_results_n50_l_path,
    'n100_s': evaluattion_results_n100_s_path,
    'n100_m': evaluattion_results_n100_m_path,
    'n100_l': evaluattion_results_n100_l_path,
    'n200_s': evaluattion_results_n200_s_path,
    'n200_m': evaluattion_results_n200_m_path,
    'n200_l': evaluattion_results_n200_l_path,
    
}

# Calculate statistics for each combination of dataset size and model complexity
evaluation_statistics = {}
for key, results in evaluation_results.items():
    map50_scores = [res['mAP@50'] for res in results]
    median_map50 = np.median(map50_scores)
    q1_map50 = np.percentile(map50_scores, 25)
    q3_map50 = np.percentile(map50_scores, 75)
    
    evaluation_statistics[key] = {
        'median': median_map50,
        'q1': q1_map50,
        'q3': q3_map50
    }

# Sort the keys to make sure the data sizes are in order (from smallest to largest dataset size)
sorted_keys = sorted(evaluation_statistics, key=lambda x: (int(x[1:x.index('_')]), x[-1]))

# Create a figure and axis for the plot
fig, ax = plt.subplots(figsize=(10, 6))

# Plot the median values and fill the area between Q1 and Q3
for model_size in ['s', 'm', 'l']:  # For each model size
    data_sizes = [int(key[1:key.index('_')]) for key in sorted_keys if key.endswith(model_size)]
    median_values = [evaluation_statistics[key]['median'] for key in sorted_keys if key.endswith(model_size)]
    q1_values = [evaluation_statistics[key]['q1'] for key in sorted_keys if key.endswith(model_size)]
    q3_values = [evaluation_statistics[key]['q3'] for key in sorted_keys if key.endswith(model_size)]

    # Plot the median line
    ax.plot(data_sizes, median_values, marker='o', label=f'Model {model_size.upper()} Median mAP@50')
    
    # Fill the area between Q1 and Q3 for the interquartile range
    ax.fill_between(data_sizes, q1_values, q3_values, alpha=0.2)

# Customize the plot
ax.set_title('Model performance: mAP')
ax.set_xlabel('Data Size')
ax.set_ylabel('mAP@50')
ax.set_xticks([25, 50, 75, 100, 125, 150, 175, 200]) 
ax.set_ylim(0.4, 1.0)  # Adjust y-axis limits to match your data
ax.legend(loc='upper left', frameon=True)  # Adjust legend to match the example
ax.grid(True, which='major', linestyle='--', linewidth=0.5)  # Adjust grid to be less prominent

# Show the plot
plt.show()